# Image Classification Using Keras Pre-trained Models

Keras Applications provide various deep learning models with pre-trianed weights using ImageNet. They can be used to predict, extract features and fine-tune.

Using TFNet in analytics-zoo, we can load these models and do predictions easily.

## Instantiate a model and construct the inference graph

When you instantiate a model for the first time, weights are downloaded automatically to ~/.keras/models/ as an h5 file. For more information, please refer to [keras document](https://keras.io/applications/).

In [1]:
from keras.applications import *
import keras.backend as K
import tensorflow as tf

K.set_learning_phase(0)
image = tf.random_uniform(shape=[224,224,3])
batch_size_tensor = tf.placeholder_with_default(32, shape=())
images = tf.train.batch(
    [image],
    batch_size = batch_size_tensor,
    num_threads = 4,
    capacity = 5 * 32)

sess = K.get_session()
kmodel = inception_v3.InceptionV3(include_top=True, weights='imagenet', input_tensor=images, input_shape=(224,224,3), pooling=None, classes=1000)
logits = kmodel.outputs

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Export the graph as a frozen inference graph

The export_tf utility function will freeze the tensorflow graph, strip unused operation according to the inputs and outputs and save it to the specified directory along with the input/output tensor names.

In [2]:
from zoo.util.tf import export_tf
export_tf(sess, "your_path", inputs=[images], outputs=[logits[0]])

INFO:tensorflow:Froze 378 variables.
Converted 378 variables to const ops.


## Load to Analytics-zoo

In [3]:
from zoo.pipeline.api.net import TFNet
model = TFNet.from_export_folder("your_path")

creating: createTFNet


## Test it on one image
![test image](./test.jpg)

In [4]:
import cv2
import numpy as np
import json
im = cv2.imread("test.jpg")
im = cv2.resize(im, (224, 224))
im = (im - 127.0) / 128.0
im = np.expand_dims(im, 0)

In [6]:
import json
with open("imagenet_class_index.json") as f:
    class_idx = json.load(f)
idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]

In [7]:
import numpy as np
result = model.predict([im])
print(idx2label[np.argmax(result, 1)[0]])

tiger_cat
